### Analyze dataset from Object Storage for Data Drift Analysis

Read set1 and set2 from Object Storage, simulate a drift and does the Data Drift analysis

In [2]:
import pandas as pd
import numpy as np
import ads

from drift_analysis import *

In [15]:
NAMESPACE = "frqap2zhtzbe"
BUCKET = "drift_input"
BUCKET_OUT = "drift_output"

REF_NAME = "reference.csv"
NEW_NAME = "current.csv"

In [4]:
# set RP auth

In [7]:
ref_url = f"oci://{BUCKET}@{NAMESPACE}/{REF_NAME}"
new_url = f"oci://{BUCKET}@{NAMESPACE}/{NEW_NAME}"

df_set1 = pd.read_csv(ref_url, storage_options={}, encoding="ISO-8859-1")
df_set2 = pd.read_csv(new_url, storage_options={}, encoding="ISO-8859-1")

In [6]:
df_set1.head()

,TravelForWork,MonthlyRate,PercentSalaryHike,CommuteLength,SalaryLevel,YearsOnJob,JobInvolvement,PerformanceRating,Gender,TrainingTimesLastYear,...,HourlyRate,MonthlyIncome,OverTime,JobSatisfaction,EducationField,JobFunction,EducationalLevel,NumCompaniesWorked,StockOptionLevel,YearsWithCurrManager
0,infrequent,19146,22,2,5640,2,2,4,Male,2,...,33,4775,No,4,Life Sciences,Software Developer,L2,6,2,2
1,none,3395,23,2,5678,23,2,4,Male,3,...,74,10748,No,3,Life Sciences,Software Developer,L1,3,1,4
2,infrequent,4510,18,15,2022,5,3,3,Female,2,...,72,4963,Yes,2,Life Sciences,Software Developer,L4,9,3,3
3,none,17071,16,25,6782,1,4,3,Female,2,...,100,13194,Yes,2,Life Sciences,Product Management,L3,4,0,0
4,infrequent,18725,23,10,1980,4,3,4,Male,4,...,96,2075,No,4,Life Sciences,Software Developer,L4,3,2,3


In [8]:
df_set2.head()

,TravelForWork,MonthlyRate,PercentSalaryHike,CommuteLength,SalaryLevel,YearsOnJob,JobInvolvement,PerformanceRating,Gender,TrainingTimesLastYear,...,HourlyRate,MonthlyIncome,OverTime,JobSatisfaction,EducationField,JobFunction,EducationalLevel,NumCompaniesWorked,StockOptionLevel,YearsWithCurrManager
0,infrequent,18597,12,3,3746,2,4,3,Male,3,...,56,8120,No,2,Marketing,Product Management,L1,3,0,2
1,often,19920,22,25,4218,1,3,4,Male,0,...,43,3161,Yes,3,Life Sciences,Software Developer,L3,3,1,0
2,often,25751,19,5,6004,0,3,3,Male,3,...,99,2976,No,3,Medical,Product Management,L1,3,0,0
3,often,2097,14,11,4114,2,3,3,Male,2,...,59,2145,No,4,Medical,Admin,L3,0,1,1
4,often,18420,12,4,2494,0,3,3,Female,2,...,70,1569,Yes,4,Medical,Product Management,L2,1,0,0


In [9]:
# simulate a drift in set2
df_set2["MonthlyIncome"] = df_set2["MonthlyIncome"] + 2000
df_set2["Age"] = df_set2["Age"] + 5

In [14]:
# a single line of code
drifts = identify_data_drift(df_set1, df_set2, do_print=True, exc_list=["Attrition"])

drifts


*** Report on evidences of Data Drift identified ***

Identified drift in column: Age
p_value: 0.0

Identified drift in column: MonthlyIncome
p_value: 0.0



[{'Column': 'Age',
  'Type': 'continuous',
  'p_value': 0.0,
  'threshold': 0.01,
  'stats': '[37.81, 9.17, 19.0, 31.0, 36.0, 43.25, 61.0],[43.36, 9.01, 24.0, 37.0, 42.0, 49.75, 66.0]',
  'was_distance_norm': 0.147,
  'delta_mean_norm': 0.147},
 {'Column': 'MonthlyIncome',
  'Type': 'continuous',
  'p_value': 0.0,
  'threshold': 0.01,
  'stats': '[6433.62, 4614.91, 1009.0, 2931.25, 4965.5, 8097.0, 19973.0],[8780.19, 5061.9, 3359.0, 4888.75, 6815.0, 10862.0, 21999.0]',
  'was_distance_norm': 0.365,
  'delta_mean_norm': 0.365}]

In [12]:
# if we want results as Pandas
result_df = pd.DataFrame(drifts)

print("*** Report on evidences of Data Drift identified ***")
print()
result_df.head()

*** Report on evidences of Data Drift identified ***



,Column,Type,p_value,threshold,stats,was_distance_norm,delta_mean_norm
0,Age,continuous,0.0,0.01,"[37.81, 9.17, 19.0, 31.0, 36.0, 43.25, 61.0],[...",0.147,0.147
1,MonthlyIncome,continuous,0.0,0.01,"[6433.62, 4614.91, 1009.0, 2931.25, 4965.5, 80...",0.365,0.365


In [18]:
# save the df_result on file on Object Storage
from datetime import datetime
now = datetime.now().strftime('%Y-%m-%d_%H_%M')

RESULT_FILE_NAME = f"drift_analysis_{now}.csv"

result_url = f"oci://{BUCKET_OUT}@{NAMESPACE}/{RESULT_FILE_NAME}"

result_df.to_csv(result_url, index=None)